In [1]:
# environment: grids with size m*n; goal / orginary grid / windy grid / cliff grid / blocking grid / start point (Can be random).
# other variation: 
# task: can be temporal discounting (R(goal)=0, R(orginary) =-1), or not (R(goal)=1, others =0).
# different learning algorithm: TD (evaluation with rand walk / e-greedy; SARSA; Q-learning)
# GUI to be added

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace



In [3]:
# Environment setup: Sutton book example 4.1
"""
# grid configuration
gsize = [3,3]
s0=[1,0] # initial state. just don't start at goal state
gw = np.zeros([gsize[0],gsize[1]]) # 0 is orginary block
gw[0,0]= 1 # set goals
gw[2,2] = 1 
acts = ['u','d','l','r']
# action and transition matrix
def state_act(state,action,gsize):
    # action is a character of either u,d,l,r (up,down,left,right)
    # state is a 1*2 tuple, marking the current position
    newstate = state[:]
    if action == 'u' or action == 0:
        newstate[1]=max(0,state[1]-1)
    elif action == 'd' or action == 1:
        newstate[1]=min(gsize[1]-1,state[1]+1)
    elif action == 'l' or action == 2:
        newstate[0]=max(0,state[0]-1)
    elif action == 'r' or action == 3:
        newstate[0]=min(gsize[0]-1,state[0]+1)
    else:
        raise ValueError('action not valid')
    
    if gw[newstate[0],newstate[1]] == -1: # means it's blocked, so remain unmoved
        newestate = state[:]
        
    return newstate    
   
# reward setup
def reward(state, gw):
    # state represents the current position; gw is the setting of grid world
    if gw[state[0],state[1]] == 1:
        R = 0
    else:
        R = -1 # with temporal discount
    return R
"""

"\n# grid configuration\ngsize = [3,3]\ns0=[1,0] # initial state. just don't start at goal state\ngw = np.zeros([gsize[0],gsize[1]]) # 0 is orginary block\ngw[0,0]= 1 # set goals\ngw[2,2] = 1 \nacts = ['u','d','l','r']\n# action and transition matrix\ndef state_act(state,action,gsize):\n    # action is a character of either u,d,l,r (up,down,left,right)\n    # state is a 1*2 tuple, marking the current position\n    newstate = state[:]\n    if action == 'u' or action == 0:\n        newstate[1]=max(0,state[1]-1)\n    elif action == 'd' or action == 1:\n        newstate[1]=min(gsize[1]-1,state[1]+1)\n    elif action == 'l' or action == 2:\n        newstate[0]=max(0,state[0]-1)\n    elif action == 'r' or action == 3:\n        newstate[0]=min(gsize[0]-1,state[0]+1)\n    else:\n        raise ValueError('action not valid')\n    \n    if gw[newstate[0],newstate[1]] == -1: # means it's blocked, so remain unmoved\n        newestate = state[:]\n        \n    return newstate    \n   \n# reward setu

In [4]:
# Environment setup: Sutton book example 6.6 cliff walk
# grid configuration
gsize = [12,4]
s0=[0,0] # initial state
gw = np.zeros([gsize[0],gsize[1]]) # 0 is orginary block
gw[-1,0]= 1 # set goals
gw[1:-2,0]= -100 # cliff
acts = ['u','d','l','r']
# action and transition matrix
def state_act(state,action,gsize):
    # action is a character of either u,d,l,r (up,down,left,right)
    # state is a 1*2 tuple, marking the current position
    newstate = state[:]
    if action == 'u' or action == 0:
        newstate[1]=max(0,state[1]-1)
    elif action == 'd' or action == 1:
        newstate[1]=min(gsize[1]-1,state[1]+1)
    elif action == 'l' or action == 2:
        newstate[0]=max(0,state[0]-1)
    elif action == 'r' or action == 3:
        newstate[0]=min(gsize[0]-1,state[0]+1)
    else:
        raise ValueError('action not valid')
    
    if gw[newstate[0],newstate[1]] == -100: # fall into the cliff? return to the initial state
        newestate = [0,0]
        
    return newstate    
   
# reward setup
def reward(state, gw):
    # state represents the current position; gw is the setting of grid world
    if gw[state[0],state[1]] == 1: # goal
        R = 0
    elif gw[state[0],state[1]] == -100: # cliff        
        R = -100 # with temporal discount
    else:
        R = -1
    return R

In [5]:
# Learner setup
A = .5 # learning rate
gamma = 1 # no temporal discount for future state
def e_greedy(state, Q): #  e-greedy
    e = 0.1 # setting recommended by example 6.5 & 6.6
    if np.random.rand(1) < e:
        action=np.random.randint(len(acts))
    else:
        Q_now = Q[state[0]][state[1]]
        #action = np.argmax(Q_now)
        allmax = [i for i, j in enumerate(Q_now) if j == max(Q_now)] # find all actions of largest Qs
        action = allmax[np.random.randint(len(allmax))] # randomly select one if there's tie; otherwise

    return action


In [8]:
# start learning
Nepis = 200 # total episode num
Q = np.zeros([gsize[0],gsize[1],len(acts)])

isQLearning=True
isSARSA=False
steps = np.zeros(Nepis)
r_epis = np.zeros(Nepis)
for k in range(Nepis):
    s = s0
    nstep = 0
    tot_r = 0
    while gw[s[0],s[1]] != 1:
        a=e_greedy(s,Q)
        s_new = state_act(s,a,gsize)
        if isSARSA == True:
            a_new = e_greedy(s_new,Q) # use the same policy as current state
        elif isQLearning == True:
            a_new = np.argmax(Q[s_new[0]][s_new[1]]) # use the best possible action        
        nstep = nstep + 1
        # print(s,s_new,acts[a])            
        pred_err = reward(s,gw) + gamma*Q[s_new[0],s_new[1],a_new] - Q[s[0],s[1],a]
        tot_r = tot_r+reward(s,gw)
        Q[s[0],s[1],a] = Q[s[0],s[1],a] + A * pred_err
        s = s_new
        
    steps[k] = nstep
    r_epis[k] = tot_r
    #print(tot_r)
    
# show the decrease of steps per episode -- average over every 10 episodes
print(np.mean(np.reshape(steps,(int(Nepis/10),10)),axis=1)) 
print(np.mean(np.reshape(r_epis,(int(Nepis/10),10)),axis=1)) 


[ 112.    51.4   38.4   27.8   22.7   22.3   15.9   17.1   16.    16.2
   15.8   14.    15.    15.9   14.4   14.7   14.5   15.8   14.1   14.6]
[-904.   -81.1  -58.2  -47.6  -62.3  -32.2  -85.2 -145.8  -55.6  -45.9
  -25.7  -14.   -84.3  -75.3  -24.3  -64.2  -24.4 -164.3  -24.   -64.1]
